In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import cv2
import matplotlib.pyplot as plt
import PIL.Image as img
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import keras
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.metrics import precision_score, recall_score, f1_score
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications import ResNet50

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/Covid19-dataset'
train_dir='/content/drive/MyDrive/Covid19-dataset/train'
test_dir='/content/drive/MyDrive/Covid19-dataset/test'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Load the VGG16 model without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Create the model
vgg16 = Sequential()
vgg16.add(base_model)
vgg16.add(Flatten())
vgg16.add(Dense(4096, activation='relu'))
vgg16.add(Dense(4096, activation='relu'))
vgg16.add(Dense(3, activation='softmax'))  # Adjust the number of classes (3 for COVID-19, Normal, Viral Pneumonia)

# Compile the model
vgg16.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

train_data = ImageDataGenerator(rescale = 1/255.0 , validation_split=0.2)
test_data = ImageDataGenerator(rescale = 1/255.0)

train_set = train_data.flow_from_directory(
                    train_dir,
                    subset = 'training',
                    batch_size=32,
                    target_size=(224,224),
                    class_mode = 'sparse'
)
val_set = train_data.flow_from_directory(
                    train_dir,
                    subset = 'validation',
                    batch_size=32,
                    target_size=(224,224),
                    class_mode = 'sparse'
)
test_set = test_data.flow_from_directory(
                    test_dir,
                    batch_size=32,
                    target_size=(224,224),
                    class_mode = 'sparse'
)

# Train the model
vgg16.fit(train_set, epochs=30, validation_data=val_set)

# Evaluate the model
loss, accuracy = vgg16.evaluate(test_set)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

Found 201 images belonging to 3 classes.
Found 50 images belonging to 3 classes.
Found 66 images belonging to 3 classes.
Epoch 1/30
7/7 [==============================] - 174s 24s/step - loss: 29.2301 - accuracy: 0.3483 - val_loss: 8.0632 - val_accuracy: 0.4400
Epoch 2/30
7/7 [==============================] - 162s 23s/step - loss: 1.8500 - accuracy: 0.6716 - val_loss: 0.9827 - val_accuracy: 0.7000
Epoch 3/30
7/7 [==============================] - 161s 23s/step - loss: 0.3497 - accuracy: 0.8856 - val_loss: 0.3129 - val_accuracy: 0.9200
Epoch 4/30
7/7 [==============================] - 161s 23s/step - loss: 0.1485 - accuracy: 0.9353 - val_loss: 0.3076 - val_accuracy: 0.9200
Epoch 5/30
7/7 [==============================] - 163s 24s/step - loss: 0.0469 - accuracy: 0.9801 - val_loss: 0.1388 - val_accuracy: 0.9200
Epoch 6/30
7/7 [==============================] - 161s 23s/step - loss: 0.0207 - accuracy: 0.9900 - val_loss: 0.2877 - val_accuracy: 0.9200
Epoch 7/30
7/7 [======================

In [ ]:
from sklearn.metrics import classification_report
import numpy as np
# Generate predictions for test set
test_set.reset()
test_data, test_labels = next(test_set)
predictions = vgg16.predict(test_data)

# Convert predicted probabilities to integer labels
predicted_labels = np.argmax(predictions, axis=1)

# Print classification report
target_names = list(test_set.class_indices.keys())
print(classification_report(test_labels, predicted_labels, target_names=target_names))

1/1 [==============================] - 18s 18s/step
                 precision    recall  f1-score   support

          Covid       1.00      1.00      1.00        13
         Normal       1.00      0.92      0.96        12
Viral Pneumonia       0.88      1.00      0.93         7

       accuracy                           0.97        32
      macro avg       0.96      0.97      0.96        32
   weighted avg       0.97      0.97      0.97        32

